# IMPORT DE LAS LIBRERÍAS, CARGA Y CONFIGURACIÓN DE LA API DE DEEPSEEK

In [ ]:
# Import de las librerías
import pandas as pd
import requests
import json
import time
import sqlite3
from typing import List, Dict, Any
import logging
from datetime import datetime
import os
import importlib

# Cargar la Api de deepseek
try:
    import config
    importlib.reload(config)  # Forzar recarga
    DEEPSEEK_API_KEY = config.DEEPSEEK_API_KEY
    DB_PATH = config.DB_PATH
    print("✅ Configuración cargada desde config.py (recargado)")
except ImportError as e:
    print(f"❌ Error: {e}")
    # Poner las credenciales directamente
    DEEPSEEK_API_KEY = "tu_api_key_real_aqui"  # ← PEGA AQUÍ TU API KEY
    DB_PATH = "restaurantes_girona.db"
    print("🔄 Usando configuración directa")

# Configuración DeepSeek API
DEEPSEEK_API_URL = "https://api.deepseek.com/v1/chat/completions"

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print(f"\n✅ Configuración lista")
print(f"📁 DB Path: {DB_PATH}")
print(f"🔑 API Key: {'✅ CONFIGURADA' if DEEPSEEK_API_KEY and DEEPSEEK_API_KEY != 'tu_api_key_real_aqui' else '❌ PENDIENTE'}")

✅ Configuración cargada desde config.py (recargado)

✅ Configuración lista
📁 DB Path: tfm_database.db
🔑 API Key: ✅ CONFIGURADA


### Establecer los parámetros del procesamiento

In [ ]:
# Configuración ajustable de los parámetros de procesamiento
BATCH_SIZE_PARAM = 3      # reviews por lote
DELAY_LOTE_PARAM = 5      # segundos entre lotes 
MAX_REVIEWS_PARAM = 12    # las reviews totales que hará el procesador (si se pone None, procesará todas las reviews pendientes)

print(f"⚙️  Configuración: {BATCH_SIZE_PARAM} batches, delay {DELAY_LOTE_PARAM}s y {MAX_REVIEWS_PARAM} reviews máximas")

# DEFINIR LAS FUNCIONES PARA ANALIZAR Y PROCESAR LAS REVIEWS

In [2]:
# Análisis exploratorio inicial de las reviews
def analizar_estado_actual():
    """Analiza las reviews existentes para entender la distribución"""
    conn = sqlite3.connect(DB_PATH)
    
    try:
        # Información básica de la tabla
        print("📊 ANÁLISIS INICIAL DE REVIEWS")
        print("=" * 50)
        
        # Total de reviews
        total_reviews = conn.execute("SELECT COUNT(*) FROM Reviews").fetchone()[0]
        print(f"📈 Total de reviews: {total_reviews}")
        
        # Distribución por idioma
        print("\n🌎 DISTRIBUCIÓN POR IDIOMA:")
        idiomas = conn.execute("SELECT idioma, COUNT(*) FROM Reviews GROUP BY idioma ORDER BY COUNT(*) DESC").fetchall()
        for idioma, count in idiomas:
            porcentaje = (count / total_reviews) * 100
            print(f"   {idioma}: {count} reviews ({porcentaje:.1f}%)")
        
        # Distribución por rating
        print("\n⭐ DISTRIBUCIÓN POR RATING:")
        ratings = conn.execute("SELECT rating, COUNT(*) FROM Reviews GROUP BY rating ORDER BY rating").fetchall()
        for rating, count in ratings:
            porcentaje = (count / total_reviews) * 100
            print(f"   {rating} estrellas: {count} reviews ({porcentaje:.1f}%)")
        
        # Reviews que necesitan procesamiento
        reviews_sin_procesar = conn.execute(
            "SELECT COUNT(*) FROM Reviews WHERE factores_queja IS NULL"
        ).fetchone()[0]
        print(f"\n🔄 Reviews pendientes de procesar: {reviews_sin_procesar}")
        
        # Ejemplos de reviews para entender el contenido
        print("\n📝 EJEMPLOS DE REVIEWS (primeras 3):")
        ejemplos = conn.execute(
            "SELECT titulo, contenido, rating, idioma FROM Reviews LIMIT 3"
        ).fetchall()
        
        for i, (titulo, contenido, rating, idioma) in enumerate(ejemplos, 1):
            print(f"\n--- Ejemplo {i} ---")
            print(f"Idioma: {idioma}")
            print(f"Rating: {rating}/5")
            print(f"Título: {titulo}")
            print(f"Contenido: {contenido[:200]}...")
            
    except Exception as e:
        print(f"❌ Error en análisis: {e}")
    finally:
        conn.close()

# Ejecutar análisis
analizar_estado_actual()

📊 ANÁLISIS INICIAL DE REVIEWS
📈 Total de reviews: 328

🌎 DISTRIBUCIÓN POR IDIOMA:
   es: 278 reviews (84.8%)
   en: 30 reviews (9.1%)
   fr: 14 reviews (4.3%)
   it: 3 reviews (0.9%)
   ru: 1 reviews (0.3%)
   nl: 1 reviews (0.3%)
   de: 1 reviews (0.3%)

⭐ DISTRIBUCIÓN POR RATING:
   1 estrellas: 218 reviews (66.5%)
   2 estrellas: 110 reviews (33.5%)

🔄 Reviews pendientes de procesar: 0

📝 EJEMPLOS DE REVIEWS (primeras 3):

--- Ejemplo 1 ---
Idioma: es
Rating: 1/5
Título: Si te van a hacer esperar haciendo cola ¿por qué se tiene que reservar?
Contenido: ¿Para qué reservar si te hacen esperar?
Si llega antes, esperas aún más. 
Por muy buena que pueda ser la comida, media hora larga de pie, a plena sol, no compensa......

--- Ejemplo 2 ---
Idioma: es
Rating: 2/5
Título: Un mal recuerdo por una situación que se podía haber solucionado con previsión
Contenido: Ya habíamos estado en este restaurante y por eso repetimos. La comida no está mal, pero, en un restaurante, valoras tanto la comi

In [3]:
# Clase procesadora actualizada para las 3 columnas
class ProcesadorReviewsCompleto:
    def __init__(self, api_key: str, db_path: str):
        self.api_key = api_key
        self.db_path = db_path
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        self.api_url = DEEPSEEK_API_URL
        
        # Factores de queja estandarizados
        self.factores_queja = [
            "servicio_lento", "mal_servicio", "precio_elevado", "calidad_comida_mala",
            "comida_fria", "comida_poco_cocinada", "limpieza_mala", "ambiente_malo",
            "tiempo_espera_excesivo", "reservas_problemas", "bebidas_malas", 
            "ubicacion_mala", "ruido_excesivo", "espacio_insuficiente", 
            "presentacion_mala", "variedad_escasa", "cantidad_insuficiente",
            "horario_inadecuado", "staff_maleducado", "atencion_mesas_mala"
        ]
    
    def get_connection(self):
        return sqlite3.connect(self.db_path)
    
    def get_reviews_pendientes(self, limit: int = None):
        """Obtiene reviews que necesitan procesamiento"""
        query = """
        SELECT id, review_id, titulo, contenido, rating, idioma
        FROM Reviews 
        WHERE titulo_es IS NULL OR contenido_es IS NULL OR factores_queja IS NULL
        ORDER BY id
        """
        if limit:
            query += f" LIMIT {limit}"
            
        try:
            with self.get_connection() as conn:
                df = pd.read_sql(query, conn)
                print(f"📊 Reviews pendientes: {len(df)}")
                return df
        except Exception as e:
            print(f"❌ Error obteniendo reviews: {e}")
            return pd.DataFrame()
    
    def llamar_api_deepseek(self, prompt: str, max_tokens: int = 2000) -> str:
        """Llama a la API de DeepSeek"""
        payload = {
            "model": "deepseek-chat",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.1,
            "max_tokens": max_tokens
        }
        
        try:
            response = requests.post(self.api_url, headers=self.headers, json=payload, timeout=60)
            response.raise_for_status()
            result = response.json()
            return result['choices'][0]['message']['content'].strip()
        except Exception as e:
            logger.error(f"❌ Error API: {e}")
            raise
    
    def traducir_texto(self, texto: str, idioma_origen: str, tipo: str = "contenido") -> str:
        """Traduce texto al español si no lo está"""
        if not texto or texto.strip() == "":
            return ""
            
        if idioma_origen.lower() == 'es':
            return texto  # Ya está en español
            
        prompt = f"""
        Traduce el siguiente texto del {idioma_origen} al español. 
        {"Es el título de una crítica de restaurante" if tipo == "titulo" else "Es el contenido de una crítica de restaurante"}
        
        Mantén el tono, significado y detalles importantes.
        
        TEXTO EN {idioma_origen.upper()}:
        {texto}
        
        TRADUCCIÓN AL ESPAÑOL:
        """
        
        return self.llamar_api_deepseek(prompt)
    
    def analizar_factores_queja(self, titulo_es: str, contenido_es: str, rating: int) -> Dict[str, int]:
        """Analiza la review y devuelve diccionario de factores de queja"""
        
        texto_completo = f"TÍTULO: {titulo_es}\nCONTENIDO: {contenido_es}\nVALORACIÓN: {rating}/5"
        
        prompt = f"""
        Analiza esta crítica de restaurante y determina qué factores de queja menciona.
        
        CRÍTICA:
        {texto_completo}
        
        CATEGORÍAS DISPONIBLES (responde con 1 si se menciona, 0 si no):
        {', '.join(self.factores_queja)}
        
        Responde ÚNICAMENTE con un JSON donde cada clave sea una categoría y el valor sea 1 o 0. Todos los campos que debes rellenar son los siguientes, con un 1 o un 0 según la queja.
        Ejemplo que debes seguir rellenando todos estos campos: {{"servicio_lento": 0, "mal_servicio": 0, "precio_elevado": 0, "calidad_comida_mala": 1, 
        "comida_fria": 0, "comida_poco_cocinada": 0, "limpieza_mala": 0, "ambiente_malo": 0, "tiempo_espera_excesivo": 0, "reservas_problemas": 0, 
        "bebidas_malas": 0, "ubicacion_mala": 0, "ruido_excesivo": 0, "espacio_insuficiente": 0, "presentacion_mala": 0, 
        "variedad_escasa": 1, "cantidad_insuficiente": 0, "horario_inadecuado": 0, "staff_maleducado": 0, "atencion_mesas_mala": 0}}
        
        Sé objetivo y basado únicamente en el texto. No inventes información.
        
        JSON:
        """
        
        respuesta = self.llamar_api_deepseek(prompt)
        
        try:
            # Limpiar respuesta
            if '```json' in respuesta:
                respuesta = respuesta.split('```json')[1].split('```')[0]
            elif '```' in respuesta:
                respuesta = respuesta.split('```')[1]
            
            factores = json.loads(respuesta.strip())
            
            # Validar que todas las categorías estén presentes
            factores_validados = {}
            for categoria in self.factores_queja:
                valor = factores.get(categoria)
                if valor in [1, '1', True, 'true']:
                    factores_validados[categoria] = 1
                else:
                    factores_validados[categoria] = 0
            
            return factores_validados
            
        except Exception as e:
            print(f"❌ Error parseando factores: {e}")
            # Devolver diccionario por defecto en caso de error
            return {categoria: 0 for categoria in self.factores_queja}
    
    def procesar_review(self, review_data: pd.Series) -> bool:
        """Procesa una review individual"""
        try:
            print(f"\n🔄 Procesando review {review_data['review_id']}")
            print(f"   Idioma: {review_data['idioma']}, Rating: {review_data['rating']}")
            
            # 1. Traducir título si es necesario
            titulo_es = self.traducir_texto(
                review_data['titulo'], 
                review_data['idioma'], 
                "titulo"
            )
            print("   ✅ Título traducido")
            
            # 2. Traducir contenido si es necesario
            contenido_es = self.traducir_texto(
                review_data['contenido'], 
                review_data['idioma'], 
                "contenido"
            )
            print("   ✅ Contenido traducido")
            
            # 3. Analizar factores de queja
            factores = self.analizar_factores_queja(titulo_es, contenido_es, review_data['rating'])
            factores_activos = [k for k, v in factores.items() if v == 1]
            print(f"   ✅ Factores identificados: {factores_activos}")
            
            # 4. Guardar en base de datos
            with self.get_connection() as conn:
                update_query = """
                UPDATE Reviews 
                SET titulo_es = ?, contenido_es = ?, factores_queja = ?
                WHERE review_id = ?
                """
                conn.execute(update_query, (
                    titulo_es, 
                    contenido_es, 
                    json.dumps(factores, ensure_ascii=False), 
                    review_data['review_id']
                ))
                conn.commit()
            
            print(f"   ✅ Review {review_data['review_id']} guardada")
            return True
            
        except Exception as e:
            print(f"❌ Error procesando review {review_data['review_id']}: {e}")
            return False
    
    def procesar_lote(self, batch_size: int = 3, delay: float = 2):
        """Procesa un lote de reviews"""
        reviews_df = self.get_reviews_pendientes(limit=batch_size)
        
        if reviews_df.empty:
            print("🎉 No hay más reviews pendientes!")
            return 0, 0
        
        procesadas = 0
        errores = 0
        
        print(f"🚀 Procesando lote de {len(reviews_df)} reviews...")
        
        for _, review in reviews_df.iterrows():
            exito = self.procesar_review(review)
            
            if exito:
                procesadas += 1
            else:
                errores += 1
            
            time.sleep(delay)  # Evitar rate limiting
        
        print(f"\n📊 Lote completado: {procesadas} exitosas, {errores} errores")
        return procesadas, errores

# Inicializar el procesador
procesador = ProcesadorReviewsCompleto(DEEPSEEK_API_KEY, DB_PATH)
print("✅ Procesador completo inicializado")

✅ Procesador completo inicializado


In [5]:
# Función de procesamiento por lotes
def procesar_todas_reviews(batch_size, delay_lotes, max_reviews):
    """
    Procesa todas las reviews pendientes
    
    Args:
        batch_size: Reviews por lote (3-5 recomendado)
        delay_lotes: Segundos entre lotes
        max_reviews: Límite máximo
    """
    total_procesadas = 0
    total_errores = 0
    lote_actual = 1
    
    print(f"🚀 INICIANDO PROCESAMIENTO COMPLETO")
    print(f"   Lote size: {batch_size}")
    print(f"   Delay: {delay_lotes}s")
    print("=" * 50)
    
    while True:
        print(f"\n📦 LOTE #{lote_actual}")
        
        procesadas, errores = procesador.procesar_lote(batch_size, delay=2)
        total_procesadas += procesadas
        total_errores += errores
        
        # Verificar si terminar
        if procesadas == 0 or (max_reviews and total_procesadas >= max_reviews):
            break
            
        # Esperar entre lotes
        if delay_lotes > 0:
            print(f"⏳ Esperando {delay_lotes} segundos...")
            time.sleep(delay_lotes)
        
        lote_actual += 1
    
    print(f"\n🎉 PROCESAMIENTO COMPLETADO")
    print(f"   Total: {total_procesadas} procesadas, {total_errores} errores")
    return total_procesadas, total_errores

### Test de conexión y preparación con la API

In [4]:
# Test de conexión y preparación
print("🔧 PREPARANDO PROCESAMIENTO COMPLETO...")

# Test de conexión API
print("\n🧪 TESTEANDO CONEXIÓN API...")
try:
    test_prompt = "Responde con 'OK' si la conexión funciona."
    respuesta = procesador.llamar_api_deepseek(test_prompt, max_tokens=10)
    print(f"✅ Conexión API: {respuesta}")
except Exception as e:
    print(f"❌ Error conexión API: {e}")

# Mostrar estado actual
print("\n📊 ESTADO ACTUAL:")
with procesador.get_connection() as conn:
    total = conn.execute("SELECT COUNT(*) FROM Reviews").fetchone()[0]
    pendientes_titulo = conn.execute("SELECT COUNT(*) FROM Reviews WHERE titulo_es IS NULL").fetchone()[0]
    pendientes_contenido = conn.execute("SELECT COUNT(*) FROM Reviews WHERE contenido_es IS NULL").fetchone()[0]
    pendientes_factores = conn.execute("SELECT COUNT(*) FROM Reviews WHERE factores_queja IS NULL").fetchone()[0]
    
    print(f"   Total reviews: {total}")
    print(f"   Pendientes titulo_es: {pendientes_titulo}")
    print(f"   Pendientes contenido_es: {pendientes_contenido}")
    print(f"   Pendientes factores_queja: {pendientes_factores}")

🔧 PREPARANDO PROCESAMIENTO COMPLETO...

🧪 TESTEANDO CONEXIÓN API...
✅ Conexión API: OK

📊 ESTADO ACTUAL:
   Total reviews: 328
   Pendientes titulo_es: 0
   Pendientes contenido_es: 0
   Pendientes factores_queja: 0


# PROCESAR Y VERIFICAR LOS RESULTADOS DE LAS REVIEWS

In [ ]:
# Procesar todas las reviews
procesadas, errores = procesar_todas_reviews(
    batch_size=BATCH_SIZE_PARAM,
    delay_lotes=DELAY_LOTE_PARAM,
    max_reviews=MAX_REVIEWS_PARAM
)

🚀 INICIANDO PROCESAMIENTO COMPLETO
🚀 INICIANDO PROCESAMIENTO COMPLETO
   Lote size: 3
   Delay: 10s

📦 LOTE #1
📊 Reviews pendientes: 0
🎉 No hay más reviews pendientes!

🎉 PROCESAMIENTO COMPLETADO
   Total: 0 procesadas, 0 errores


In [ ]:
# Verificación de resultados
print("🔍 VERIFICANDO RESULTADOS...")

with procesador.get_connection() as conn:
    # Estadísticas finales
    total = conn.execute("SELECT COUNT(*) FROM Reviews").fetchone()[0]
    con_titulo_es = conn.execute("SELECT COUNT(*) FROM Reviews WHERE titulo_es IS NOT NULL").fetchone()[0]
    con_contenido_es = conn.execute("SELECT COUNT(*) FROM Reviews WHERE contenido_es IS NOT NULL").fetchone()[0]
    con_factores = conn.execute("SELECT COUNT(*) FROM Reviews WHERE factores_queja IS NOT NULL").fetchone()[0]
    
    # Distribución de factores
    factores_data = conn.execute("SELECT factores_queja FROM Reviews WHERE factores_queja IS NOT NULL").fetchall()
    conteo_factores = {}
    for (factores_json,) in factores_data:
        try:
            factores = json.loads(factores_json)
            for factor, valor in factores.items():
                if valor == 1:
                    conteo_factores[factor] = conteo_factores.get(factor, 0) + 1
        except:
            continue
    
    # Ejemplos de resultados
    ejemplos = conn.execute("""
    SELECT idioma, titulo, titulo_es, contenido_es, factores_queja, rating 
    FROM Reviews 
    WHERE titulo_es IS NOT NULL 
    LIMIT 3
    """).fetchall()

print(f"\n📊 RESULTADOS FINALES:")
print(f"   Total reviews: {total}")
print(f"   Con titulo_es: {con_titulo_es} ({con_titulo_es/total*100:.1f}%)")
print(f"   Con contenido_es: {con_contenido_es} ({con_contenido_es/total*100:.1f}%)")
print(f"   Con factores_queja: {con_factores} ({con_factores/total*100:.1f}%)")

print(f"\n🏷️ FACTORES DE QUEJA MÁS COMUNES:")
for factor, count in sorted(conteo_factores.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"   {factor}: {count} veces")

print(f"\n📝 EJEMPLOS PROCESADOS:")
for i, (idioma, titulo_orig, titulo_es, contenido_es, factores, rating) in enumerate(ejemplos, 1):
    print(f"\n--- Ejemplo {i} ---")
    print(f"Idioma original: {idioma}")
    print(f"Rating: {rating}/5")
    print(f"Título original: {titulo_orig}")
    print(f"Título español: {titulo_es}")
    print(f"Contenido español: {contenido_es[:100]}...")
    print(f"Factores: {json.loads(factores) if factores else {}}")

🔍 VERIFICANDO RESULTADOS...

📊 RESULTADOS FINALES:
   Total reviews: 328
   Con titulo_es: 328 (100.0%)
   Con contenido_es: 328 (100.0%)
   Con factores_queja: 328 (100.0%)

🏷️ FACTORES DE QUEJA MÁS COMUNES:
   mal_servicio: 226 veces
   atencion_mesas_mala: 193 veces
   calidad_comida_mala: 182 veces
   staff_maleducado: 126 veces
   servicio_lento: 101 veces
   precio_elevado: 91 veces
   tiempo_espera_excesivo: 87 veces
   cantidad_insuficiente: 63 veces
   ambiente_malo: 39 veces
   comida_fria: 37 veces

📝 EJEMPLOS PROCESADOS:

--- Ejemplo 1 ---
Idioma original: es
Rating: 1/5
Título original: Si te van a hacer esperar haciendo cola ¿por qué se tiene que reservar?
Título español: Si te van a hacer esperar haciendo cola ¿por qué se tiene que reservar?
Contenido español: ¿Para qué reservar si te hacen esperar?
Si llega antes, esperas aún más. 
Por muy buena que pueda se...
Factores: {'servicio_lento': 0, 'mal_servicio': 0, 'precio_elevado': 0, 'calidad_comida_mala': 0, 'comida_fria